In [29]:
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

data=pd.read_csv("/kaggle/input/loan-eligible-dataset/loan-train.csv")
df_train=data.copy()



In [30]:
# Size Of the Data 
df_train.shape


In [31]:
df_train.isnull().values.any()

In [32]:
df_train.isnull().sum()


In [33]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data(df_train)



In [34]:
# Duplicated data
df_train[df_train.duplicated() == True]


In [35]:

#check for null
df_train.isnull().sum()


In [36]:

df_train_clean=df_train.copy()
df_train_clean=df_train_clean.dropna()
df_train_clean.drop(['Loan_ID'],inplace=True,axis=1)
df_train_clean.head()



In [37]:
#data preprocessing
df_train_num=df_train_clean.copy()
Gender= {'Male':0,'Female':1}
Married= {'Yes':1,'No':0}
Education= {'Graduate':1,'Not Graduate':0}
Self_Employed= {'Yes':1,'No':0}
Dependents= {'0':0,'1':1,'2':2,'3+':3}
Loan_Status= {'Y':1,'N':0}
Rural={'Rural':1,'Urban':0,'Semiurban':0}
Urban={'Rural':0,'Urban':1,'Semiurban':0}
Semiurban={'Rural':0,'Urban':0,'Semiurban':1}
df_train_num.head()



In [38]:

df_train_num["Gender"]=df_train_num["Gender"].map(Gender)
df_train_num["Married"]=df_train_num["Married"].map(Married)
df_train_num["Education"]=df_train_num["Education"].map(Education)
df_train_num["Dependents"]=df_train_num["Dependents"].map(Dependents)
df_train_num["Self_Employed"]=df_train_num["Self_Employed"].map(Self_Employed)
df_train_num["Loan_Status"]=df_train_num["Loan_Status"].map(Loan_Status)
df_train_num["Rural_Area"]=df_train_num["Property_Area"].map(Rural)
df_train_num["Urban_Area"]=df_train_num["Property_Area"].map(Urban)
df_train_num["Semiurban_Area"]=df_train_num["Property_Area"].map(Semiurban)

In [39]:

df_train_num.drop(['Property_Area'],inplace=True,axis=1)
df_train_num.head()


In [40]:
#removing the outliers
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)
df_out=df_train_num.copy()
clf.fit_predict(df_out)
df_scores = clf.negative_outlier_factor_


In [41]:
np.sort(df_scores)[0:70]



In [42]:

threshold = np.sort(df_scores)[14]
threshold

Outlier_df= df_out[df_scores < threshold]
indexs=Outlier_df.index
Outlier_df

for i in indexs:
    df_train_num.drop(i, inplace = True, axis = 0)

In [43]:


y=df_train_num['Loan_Status']
print('y shape :',y.shape)

X=df_train_num.drop('Loan_Status',axis=1)
print('X shape :',X.shape)

In [44]:
#Normalization
X = (X - np.min(X)) / (np.max(X) - np.min(X)).values
X.head()


from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)



In [45]:
#KNN
from sklearn.neighbors import KNeighborsClassifier as KNC
knn = KNC()
knn_model=knn.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)
accuracy_score(y_test, y_pred_knn)

#KNN heatmap
cm = confusion_matrix(y_test, y_pred_knn)
sns.heatmap(cm, annot=True)


In [46]:
#Random forest
from sklearn.ensemble import RandomForestClassifier as rf
rf_model = rf().fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
accuracy_score(y_test, y_pred_rf)


In [47]:

#rf heatmap
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True)

In [48]:


#SVM
from sklearn.svm import SVC as svm
svm_model = svm(kernel = "linear").fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
accuracy_score(y_test, y_pred_svm)

In [49]:
#SVM heatmap
cm = confusion_matrix(y_test, y_pred_svm)
sns.heatmap(cm, annot=True)
